In [1]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_table_manager import TableManager
import yfinance as yf
from lab_finalytics_database import FinalyticsDB

In [2]:
with open("cfg_connections.yaml","r") as file:
    config=yaml.safe_load(file)
    catalog_uri = config['docker_env']['catalog_uri'] 
    warehouse = config['docker_env']['warehouse']     # Minio Address to Write to
    storage_uri = config['docker_env']['storage_uri'] # Minio IP address from docker inspec

# Configure Spark with necessary packages and Iceberg/Nessie settings
conf = (
    pyspark.SparkConf()
        .setAppName('finalytics_app')
        # Include necessary packages
        .set('spark.jars.packages',
             'org.postgresql:postgresql:42.7.3,'
             'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,'
             'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,'
             'software.amazon.awssdk:bundle:2.24.8,'
             'software.amazon.awssdk:url-connection-client:2.24.8')
        # Enable Iceberg and Nessie extensions
        .set('spark.sql.extensions', 
             'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,'
             'org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        # Configure Nessie catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', catalog_uri)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        # Set Minio as the S3 endpoint for Iceberg storage
        .set('spark.sql.catalog.nessie.s3.endpoint', storage_uri)
        .set('spark.sql.catalog.nessie.warehouse', warehouse)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')       
)   

# Start Spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()  
# Create the "sales" namespace
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.raw;").show()


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dcf8406f-a94f-4b16-954f-2ba9b4e66692;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.0 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.77.1 in central
	found software.amazon.awssdk#bundle;2.24.8 in central
	found software.amazon.awssdk#url-connection-client;2.24.8 in central
	found software.amazon.awssdk#utils;2.24

++
||
++
++



In [3]:
def fetch_yfinance_record(symbol_date_pairs):
    try:
        symbol, start_date = symbol_date_pairs
        # Fetch stock data using yfinance
        quote = yf.Ticker(symbol)
        current_date = date.today()
        hist = quote.history(start=start_date, end=current_date)

        # Reset index to include Date as a column and format it
        hist.reset_index(inplace=True)
        # hist['Date'] = hist['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
        # hist['Date'] = hist['Date'].dt.strftime('%Y-%m-%d').date()
        hist["Date"] = hist["Date"].dt.date 
        
        # Add symbol and import_time to each row
        record_list = [
            tuple(row) + (symbol, import_time) for row in hist.itertuples(index=False)
        ]
        
        # print(record_list)
        return record_list

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return []  # Return an empty list on error

In [4]:
def parallel_fetch_yfinance_record(symbol_date_pairs, record_schema):    
    # Distribute (symbol, start_date) pairs across Spark workers
    record_rdd = spark.sparkContext.parallelize(symbol_date_pairs)
    
    # Fetch data in parallel
    mapped_record_rdd = record_rdd.flatMap(fetch_yfinance_record)

    # Convert RDD to DataFrame
    result_df = spark.createDataFrame(mapped_record_rdd, schema=record_schema)

    # Show or save the results
    # result_df.show()
    return result_df

In [5]:
def load_raw_eod_yfinance(symbol_date_pairs, sink_table, schema_config_file):
    table_manager=TableManager(schema_config_file)
    regd_struct_type=table_manager.get_struct_type(sink_table)   
    
    # regd_column_list = table_manager.get_column_list(sink_table)
    create_table_script = table_manager.get_create_table_query(sink_table)
    spark.sql(create_table_script)
    
    df_raw_eod_yfinance=parallel_fetch_yfinance_record(symbol_date_pairs, regd_struct_type)
    # df_raw_eod_yfinance = df_raw_eod_yfinance.withColumn("date", to_date("date", "yyyy-MM-dd")).withColumn("import_time", to_timestamp("import_time", "yyyy-MM-dd HH:mm:ss"))
  
    # df_raw_eod_yfinance.writeTo(sink_table).append()
    df_raw_eod_yfinance.write.mode("overwrite").saveAsTable(sink_table)    
    # df_raw_eod_yfinance.writeTo(sink_table).overwritePartitions()
    
    print(f"{sink_table} has been loaded")
    

In [30]:
  
# symbol_start_date_pairs = [
#     ('AAPL', '2024-12-10'),
#     ('MSFT', '2024-12-10'),
#     ('GOOGL', '2024-12-10'),
# ]

conn_config_file='cfg_connections.yaml'
finalytics=FinalyticsDB(conn_config_file)

query="select symbol, start_date from fin.vw_etl_stock_eod_start_date limit 30"
symbol_start_date_pairs=finalytics.get_symbol_start_date_pairs(query)
finalytics_url=finalytics.jdbc_url
finalytics_driver=finalytics.driver

regd_schema_config_file='cfg_registered_table_schemas.yaml'
sink_table='nessie.raw.stock_eod_yfinance'

import_time = datetime.now().isoformat()
load_raw_eod_yfinance(symbol_start_date_pairs, sink_table, regd_schema_config_file)



Could not get exchangeTimezoneName for ticker 'KSM' reason: 'chart' (2 + 2) / 4]
$KSM: possibly delisted; no timezone found
Error fetching data for KSM: Can only use .dt accessor with datetimelike values


nessie.raw.stock_eod_yfinance has been loaded


In [28]:
spark.sql('select count(*) from nessie.raw.stock_eod_yfinance').show()

+--------+
|count(1)|
+--------+
|      17|
+--------+



In [29]:
df=spark.read.table(sink_table)          
pg_table = "stage.stock_eod_quote_yahoo"  # Replace with the PostgreSQL table name

query = f"truncate table {pg_table};"
finalytics.execute_sql_script(query)

# # Write Delta table DataFrame to PostgreSQL
df.write.jdbc(url=finalytics_url, table=pg_table, mode="append", properties={"driver": finalytics_driver})

query = "call fin.usp_load_stock_eod();"
finalytics.execute_sql_script(query)



In [19]:
spark.sql('select import_time from nessie.raw.stock_eod_yfinance order by import_time desc limit 1').show()

+--------------------+
|         import_time|
+--------------------+
|2024-12-25T04:04:...|
+--------------------+

